In [3]:
import pandas as pd
import numpy as np
import scipy

from scipy import stats
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
data = pd.read_csv('ab_browser_test.csv')
data.head()

# Описание данных:
# userID: уникальный идентификатор пользователя
# browser: браузер, который использовал userID
# slot: в каком статусе пользователь участвовал в исследовании(exp = видел измененную страницу, control = видел неизменную страницу)
# n_clicks: количество кликов, которые пользоваль совершил за n_queries
# n_queries: количество запросов, который совершил userID, пользуясь браузером browser
# n_nonclk_queries: количество запросов пользователя, в которых им не было совершено ни одного клика

,userID,browser,slot,n_clicks,n_queries,n_nonclk_queries
0,1,Browser #2,exp,23,32,19
1,3,Browser #4,exp,3,4,2
2,5,Browser #4,exp,29,35,16
3,6,Browser #4,control,12,6,0
4,7,Browser #4,exp,54,68,30


In [5]:
# 1. Основная метрика, на которой мы сосредоточимся в этой работе, — это количество пользовательских кликов на web-странице 
# в зависимости от тестируемого изменения этой страницы.
# Посчитайте, насколько в группе exp больше пользовательских кликов по сравнению с группой control в процентах от числа кликов 
# в контрольной группе. Полученный процент округлите до третьего знака после точки.

exp = data[data['slot']=='exp']
control = data[data['slot']=='control']
print (round((sum(exp.n_clicks)*1.0/sum(control.n_clicks)-1)*100,3))

1.614


In [6]:
# 2. Давайте попробуем посмотреть более внимательно на разницу между двумя группами (control и exp) относительно количества 
# пользовательских кликов. Для этого постройте с помощью бутстрепа 95% доверительный интервал 
# для средних значений и медиан количества кликов в каждой из двух групп. 

data.describe()

,userID,n_clicks,n_queries,n_nonclk_queries
count,566134.000000,566134.000000,566134.000000,566134.000000
mean,301377.214027,11.431658,10.720524,4.703987
std,175526.333101,20.251494,16.262827,8.109958
min,1.000000,0.000000,1.000000,0.000000
25%,148627.250000,1.000000,2.000000,1.000000
50%,299362.500000,4.000000,5.000000,2.000000
75%,455698.750000,13.000000,13.000000,6.000000
max,603137.000000,863.000000,665.000000,581.000000


In [7]:
exp_mean = np.mean(exp.n_clicks)
exp_med = np.median(exp.n_clicks)
print "EXP: mean = %.4f; median = %.4f" % (exp_mean, exp_med)

c_mean = np.mean(control.n_clicks)
c_med = np.median(control.n_clicks)
print "CONTROL: mean = %.4f; median = %.4f" % (c_mean, c_med)

EXP: mean = 11.5840; median = 5.0000
CONTROL: mean = 11.2809; median = 4.0000


In [42]:
def get_bootstrap_samples(data, n_samples):
    # генерируются случайные подвыборки размера n_samples X len(data): 500 Х 15000 со значениями от 0 до len(data)
    # формируем массив с индексами значений, которые необходимо отобрать
    indices = np.random.randint(0, len(data), (n_samples, len(data))) 
    # передаем массив индексов в массив исходных значений для формирования псевдовыборок
    samples = data[indices]
    return samples

In [43]:
def stat_intervals(stat, alpha=0.05):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [11]:
np.random.seed(123)

exp_n_clicks = np.array(exp.n_clicks.values)
c_n_clicks = np.array(control.n_clicks.values)

exp_median_scores = np.median(get_bootstrap_samples(exp_n_clicks, 500), axis=1)
c_median_scores = np.median(get_bootstrap_samples(c_n_clicks, 500), axis=1)

print "95% confidence interval for Exp group:",  stat_intervals(exp_median_scores)
print "95% confidence interval for Control group:",  stat_intervals(c_median_scores)

95% confidence interval for Exp group: [ 5.  5.]
95% confidence interval for Control group: [ 4.  4.]


In [15]:
exp_n_clicks[:20]

array([23,  3, 29, 54,  6,  2,  7,  4,  2, 21, 10, 25, 72,  4,  7, 22,  3,
        1, 39, 32])

In [24]:
median_scores = map(lambda x: x[0] - x[1], zip(exp_median_scores, c_median_scores))
print ("95% confidence interval for difference median:",  stat_intervals(median_scores))

('95% confidence interval for difference median:', array([ 1.,  1.]))


In [25]:
np.random.seed(123)

exp_mean_scores = np.mean(get_bootstrap_samples(exp_n_clicks, 500), axis=1)
c_mean_scores = np.mean(get_bootstrap_samples(c_n_clicks, 500), axis=1)

print "95% confidence interval for Exp group:",  stat_intervals(exp_mean_scores)
print "95% confidence interval for Control group:",  stat_intervals(c_mean_scores)

95% confidence interval for Exp group: [ 11.51651342  11.66053004]
95% confidence interval for Control group: [ 11.20560816  11.35528713]


In [26]:
mean_scores = map(lambda x: x[0] - x[1], zip(exp_mean_scores, c_mean_scores))
print ("95% confidence interval for difference mean:",  stat_intervals(mean_scores))

('95% confidence interval for difference mean:', array([ 0.20274804,  0.40479205]))


In [27]:
# 4. t-критерий Стьюдента имеет множество достоинств, и потому его достаточно часто применяют в AB экспериментах. 
# Иногда его применение может быть необоснованно из-за сильной скошенности распределения данных.
# Давайте постараемся понять, когда t-критерий можно применять и как это проверить на реальных данных.
# Поскольку мы не знаем истинного распределения генеральной совокупности, мы можем применить бутстреп, 
# чтобы понять, как распределены среднее значение и выборочная дисперсия. 
# Для этого:
#  - Получите из данных n_boot_samples псевдовыборок.
np.random.seed(0)
n_boot_samples = 500
c_samples = get_bootstrap_samples(c_n_clicks, n_boot_samples)

In [28]:
#  - По каждой из этих выборок посчитайте среднее и сумму квадратов отклонения от выборочного среднего (control_boot_chi_squared)
control_boot_mean = np.mean(c_samples, axis=1)
control_boot_chi_squared = np.var(c_samples,axis=1) 

In [1]:
#  - Для получившегося вектора сумм квадратов отклонения от выборочного среднего постройте qq-plot с помощью 
# scipy.stats.probplot для хи-квадрат распределения с помощью команды:

scipy.stats.probplot(control_boot_chi_squared, dist="chi2", sparams=(500-1), plot=plt)

In [2]:
# Для получившегося вектора средних значений из n_boot_samples постройте q-q plot с помощью 
# scipy.stats.probplot для нормального распределения

scipy.stats.probplot(control_boot_mean, plot=plt)

In [ ]:
# 5. Одним из возможных аналогов t-критерия, которым можно воспрользоваться, является тест Манна-Уитни. 
# На достаточно обширном классе распределений он является асимптотически более эффективным, чем t-критерий, 
# и при этом не требует параметрических предположений о характере распределения.

In [32]:
# Разделите выборку на две части, соответствующие control и exp группам. 
# Преобразуйте данные к виду, чтобы каждому пользователю соответствовало суммарное значение его кликов. 
n_click_control_by_user = data[data['slot']=='control'].groupby(by='userID')['n_clicks'].agg(np.sum)
n_click_exp_by_user = data[data['slot']=='exp'].groupby(by='userID')['n_clicks'].agg(np.sum)

In [33]:
n_click_control_by_user.describe()

count    284392.000000
mean         11.287332
std          20.289501
min           0.000000
25%           1.000000
50%           4.000000
75%          13.000000
max         863.000000
Name: n_clicks, dtype: float64

In [34]:
n_click_exp_by_user.describe()

count    281450.000000
mean         11.589352
std          20.225421
min           0.000000
25%           1.000000
50%           5.000000
75%          13.000000
max         776.000000
Name: n_clicks, dtype: float64

In [35]:
# С помощью критерия Манна-Уитни проверьте гипотезу о равенстве средних этих двух выборок. 
# Что можно сказать о получившемся значении достигаемого уровня значимости? Нулевая гипотеза отвергается, и это свидетельствует 
# о статической значимости отличий средних двух выборок, а значит и самих выборок.

stats.mannwhitneyu(n_click_exp_by_user, n_click_control_by_user)

MannwhitneyuResult(statistic=38901259929.0, pvalue=4.3471471887604393e-75)

In [36]:
# 6. Проверьте, для какого из браузеров наиболее сильно выражено отличие между количеством кликов в контрольной и эксперимен.группах.
# Для этого примените для каждого из срезов (по каждому из уникальных значений столбца browser) критерий Манна-Уитни между 
# control и exp группами и сделайте поправку Холма-Бонферрони на множественную проверку с α=0.05.
# Какое заключение можно сделать исходя из полученных результатов ?

browsers = data.browser.unique()
print browsers

['Browser #2' 'Browser #4' 'Browser #14' 'Browser #17' 'Browser #20'
 'Browser #22']


In [39]:
pvals = []
for b in browsers:
    e = exp[exp.browser==b]
    c = control[control.browser==b]
    pval = stats.mannwhitneyu(e.n_clicks, c.n_clicks).pvalue
    pvals.append(pval)
    
print (pvals)

[0.027243860419724101, 0.40760617658454984, 0.0, 0.037400601430108017, 0.45366656388735377, 0.25740551522100319]


In [40]:
# В качестве ответа введите количество незначимых изменений с точки зрения результатов, полученных после введения коррекции.
# 5
import statsmodels.stats.multitest as smm

rej, pval_corr = smm.multipletests(pvals, alpha=0.05/6, method='holm')[:2]
print rej, pval_corr

[False False  True False False False] [ 0.1362193   0.81521235  0.          0.14960241  0.81521235  0.77221655]


In [48]:
# 7. Для каждого браузера в каждой из двух групп (control и exp) посчитайте долю запросов, 
# в которых пользователь не кликнул ни разу. Это можно сделать, поделив сумму значений n_nonclk_queries на сумму значений 
# n_queries. Умножив это значение на 100, получим процент некликнутых запросов, который можно легче проинтерпретировать.
# Сходятся ли результаты проведенного Вами анализа с показателем процента некликнутых запросов ?
# Тестируемое изменение приводит к статистически значимому отличию только для одного браузера - Browser #14.
# Для него на основе данных о некликнутых запросах можно закллючить, что тестируемое изменение влияет положительно: 
# доля некликнутых запросов снизилась с 57% до 43%

exp_perc = []
c_perc = []
for b in browsers:
    e = exp[exp.browser==b]
    c = control[control.browser==b]
    perc_e = (np.sum(e.n_nonclk_queries)*1./np.sum(e.n_queries))*100
    perc_c = (np.sum(c.n_nonclk_queries)*1./np.sum(c.n_queries))*100
    exp_perc.append(perc_e)
    c_perc.append(perc_c)
    
print ('EXPERIMENT:' + ' ' + str(exp_perc))
print ('CONTROL:' + ' ' + str(c_perc))

EXPERIMENT: [44.982746948554706, 45.14294190358467, 43.755617361273295, 36.93741284866483, 38.97737648371716, 39.85394721969546]
CONTROL: [45.96274717919465, 46.97092963514274, 57.59041136008114, 36.29936674628208, 40.540484743383296, 40.593976593513354]
